In [ ]:
'''pitchfx_data_collection.py

David Christiansen
1/18/16

1. Collect pitcherf/x data from gd2.mlb.com.
2. Allocate data for each batter --- write all batter-specific data to a file.

'''

In [28]:
import requests, re
from os import listdirt
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from datetime import datetime
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook



Web-Scraping and Data Acquisition

In [ ]:
# Get the list of URLs for the range of specified dates
base_url = 'http://gd2.mlb.com/components/game/mlb/'
year = '2014'

#GameDay - 404 Not Found
# base + year
# http://gd2.mlb.com/components/game/mlb/year_2014/ 

# base + year + month
# http://gd2.mlb.com/components/game/mlb/year_2014/month_06/
# http://gd2.mlb.com/components/game/mlb/year_2014/month_06/day_11/
# http://gd2.mlb.com/components/game/mlb/year_2014/month_06/day_11/gid_2014_06_11_slnmlb_tbamlb_1/
# http://gd2.mlb.com/components/game/mlb/year_2014/month_06/day_11/gid_2014_06_11_slnmlb_tbamlb_1/inning/
#http://gd2.mlb.com/components/game/mlb/year_2014/month_06/day_11/gid_2014_06_11_slnmlb_tbamlb_1/inning/inning_all.xml

r = requests.get(base_url + 'year_' + year)
r.text
soup1 = BeautifulSoup(r.text, 'html')

In [ ]:

months = [ re.sub(' ', '', t.text) for t in soup1.select('a') if t.text.startswith(' month') ]


for month in months:
    print month
    r = requests.get(base_url + 'year_' + year + '/' + month)
    #print r.text
    soup2 = BeautifulSoup(r.text, 'html')
    days = [ re.sub(' ', '', d.text) for d in soup2.select('a') if d.text.startswith(' day')]
    for day in days:
        s = requests.get(base_url + 'year_' + year + '/' + month  + day)
        soup3 = BeautifulSoup(s.text, 'html')
        games = [ re.sub(' ', '', g.text) for g in soup3.select('a') if g.text.startswith(' gid')]
        if games != []:
            for game in games:
                t = requests.get(base_url + 'year_' + year + '/' + month  + day + game + 'inning/inning_all.xml')
                #soup4 = BeautifulSoup(t.text, 'xml')
                # store the game in a file
                #print t.text
                with open("web_page_data/" + re.sub('/', '', game) + ".txt", "wb") as file:    
                    file.write(t.text.encode('utf-8'))

Feature Extraction and Data Alocation

In [9]:
# feature extraction function

def feature_extract(pitch):
    batter = re.findall('batter="(.*?)"', str(at_bat))[0]
    pitcher = re.findall('pitcher="(.*?)"', str(at_bat))[0]
    ax = re.findall('ax="(.*?)"', str(pitch))[0]
    ay = re.findall('ay="(.*?)"', str(pitch))[0]
    az = re.findall('az="(.*?)"', str(pitch))[0]
    break_angle = re.findall('break_angle="(.*?)"', str(pitch))[0]
    break_length = re.findall('break_length="(.*?)"', str(pitch))[0]
    break_y = re.findall('break_y="(.*?)"', str(pitch))[0]
    cc = re.findall('cc="(.*?)"', str(pitch))[0]
    des = re.findall('des="(.*?)"', str(pitch))[0]
    end_speed = re.findall('end_speed="(.*?)"', str(pitch))[0]
    #event_num = re.findall('event_num="(.*?)"', str(pitch))[0]
    ID = re.findall(' id="(.*?)"', str(pitch))[0]
    mt = re.findall('mt="(.*?)"', str(pitch))[0]
    nasty = re.findall('nasty="(.*?)"', str(pitch))[0]
    pfx_x = re.findall('pfx_x="(.*?)"', str(pitch))[0]
    pfx_z = re.findall('pfx_z="(.*?)"', str(pitch))[0]
    pitch_type = re.findall('pitch_type="(.*?)"', str(pitch))[0]
    #play_guid = re.findall('play_guid="(.*?)"', str(pitch))[0]
    px = re.findall('px="(.*?)"', str(pitch))[0]
    pz = re.findall('pz="(.*?)"', str(pitch))[0]
    spin_dir = re.findall('spin_dir="(.*?)"', str(pitch))[0]
    spin_rate = re.findall('spin_rate="(.*?)"', str(pitch))[0]
    break_y = re.findall('break_y="(.*?)"', str(pitch))[0]
    start_speed = re.findall('start_speed="(.*?)"', str(pitch))[0]
    sv_id = re.findall('sv_id="(.*?)"', str(pitch))[0]
    sz_bot = re.findall('sz_bot="(.*?)"', str(pitch))[0]
    sz_top = re.findall('sz_top="(.*?)"', str(pitch))[0]
    tfs = re.findall('tfs="(.*?)"', str(pitch))[0]

    tfs_zulu = re.findall('tfs_zulu="(.*?)"', str(pitch))[0]
    tfs_zulu = re.sub('T', ' ', tfs_zulu)
    tfs_zulu = re.sub('Z', '', tfs_zulu)

    Type = re.findall(' type="(.*?)"', str(pitch))[0]
    type_confidence = re.findall(' type_confidence="(.*?)"', str(pitch))[0]
    vx0 = re.findall(' vx0="(.*?)"', str(pitch))[0]
    vy0 = re.findall(' vy0="(.*?)"', str(pitch))[0]
    vz0 = re.findall(' vz0="(.*?)"', str(pitch))[0]
    x = re.findall(' x="(.*?)"', str(pitch))[0]
    x0 = re.findall(' x0="(.*?)"', str(pitch))[0]
    y = re.findall(' y="(.*?)"', str(pitch))[0]
    y0 = re.findall(' y0="(.*?)"', str(pitch))[0]
    z0 = re.findall(' z0="(.*?)"', str(pitch))[0]
    zone = re.findall('zone="(.*?)"', str(pitch))[0]

    profile = [batter, pitcher, ax, ay, az, break_angle, break_length, break_y,\
               des, end_speed, ID, mt, nasty, pfx_x, pfx_z, pitch_type,\
               px, pz, spin_dir, spin_rate, start_speed, sv_id, sz_bot, sz_top, tfs,\
               tfs_zulu, Type, type_confidence, vx0, vy0, vz0, x, x0, y, y0, z0, zone]
    return profile


#profile2 = feature_extract(str(pitch))
columns = ['batter', 'pitcher', 'ax', 'ay', 'az', 'break_angle', 'break_length', 'break_y',\
               'des', 'end_speed', 'ID', 'mt', 'nasty', 'pfx_x', 'pfx_z', 'pitch_type',\
               'px', 'pz', 'spin_dir', 'spin_rate', 'start_speed', 'sv_id', 'sz_bot', 'sz_top', 'tfs',\
               'tfs_zulu', 'Type', 'type_confidence', 'vx0', 'vy0', 'vz0', 'x', 'x0', 'y', 'y0', 'z0', 'zone']

# deleted event_num, play_guid

In [10]:
opening_date = datetime.strptime('2014 05 02', '%Y %m %d')
files = [ file for file in listdir('web_page_data') if file.endswith('.txt') and \
            datetime.strptime( re.findall('gid_(.{10})', file)[0].replace('_', ' '), '%Y %m %d' ) \
             >= opening_date ]

i = 0
bad_data = []
print len(files)

for file in files:
    with open( 'web_page_data/'+file, 'rb') as f:
        html_content = f.read()
    soup = BeautifulSoup(html_content, 'html')
    if i%100==0: print i, 'of', len(files)
    i += 1
    for at_bat in soup.findAll('atbat'):
        x = []
        batter = re.findall('batter="(.+?)"', str(at_bat))
        pitcher = re.findall('pitcher="(.+?)"', str(at_bat))
        for pitch in at_bat.findAll('pitch'):
            try:
                x.append(feature_extract(pitch))
            except:
                bad_data.append([pitch])
                continue
        at_bat_data = pd.DataFrame(x)
        at_bat_data.columns = columns
        batter_file = str(batter[0])+'.csv'
        if batter_file in listdir('batter_data'):
            print 'True'
            at_bat_data.to_csv('batter_data/'+batter_file, header=False, mode='a', index=False)
        else:
            print 'False'
            at_bat_data.to_csv('batter_data/'+batter_file, header=True, mode='a', index=False)


2
0 of 2
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
False
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [ ]:
bad_data

In [16]:
#listdir('batter_data')

In [12]:
batter = pd.read_csv('batter_data/136767.csv', header=0)

In [39]:
batter[['des', 'zone']]



,des,zone
0,Swinging Strike,11
1,Called Strike,1
2,Foul,11
3,Ball,12
4,Foul,2
5,"In play, out(s)",12


In [ ]:
class ColumnSelectTransformer(sk.base.BaseEstimator, sk.base.TransformerMixin):
    def __init__(self, c1, c2):
        self.c1 = c1
        self.c2 = c2
        return

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        self.transformed_df = X[[self.c1, self.c2]]
        return self.transformed_df 

In [ ]:
x = ColumnSelectTransformer('latitude', 'longitude')
X = x.transform(d)
y = list(d.stars)
X.head()

In [ ]:

#print X
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=40, weights='uniform')

neigh.fit(X, y)

with open('question2.dill', 'wb') as f:
    dill.dump(neigh, f)
with open('../question2.dill', 'wb') as f:
    dill.dump(neigh, f)

#KNeighborsRegressor(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=1)
print(neigh.predict([35,-110]))
#[ 0.5]

In [ ]:
output_notebook()



In [31]:
x = [ 1, 2, 3, 4, 5]
y = [ 1, 4, 9, 16, 25]


# output to static HTML file
#output_file("lines.html", title="line plot example")

# create a new plot with a title and axis labels
p = figure(title="simple line example", x_axis_label='x', y_axis_label='y')

# add a line renderer with legend and line thickness
p.line(x, y, legend="Temp.", line_width=2)

# show the results


NameError: name 'session' is not defined